In [ ]:


# Install required package
!pip install python-docx

# === Imports ===
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from docx import Document
from docx.shared import Inches
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# === Mount Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# Paths and parameters
input_path = 'path'
output_path = 'path'
os.makedirs(output_path, exist_ok=True)

EPOCHS = 1
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
FINE_TUNE_AT = 50
IMG_SIZE = (224, 224)

# === Word Document Initialization ===
doc = Document()
doc.add_heading('DenseNet169 (Fine-Tuned) Report with ROC & Grad-CAM', 0)

# === Data Processing ===
def process_data(input_path, img_size, batch_size):
    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.densenet.preprocess_input,
        validation_split=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest"
    )
    train_gen = datagen.flow_from_directory(input_path, target_size=img_size, batch_size=batch_size,
                                            class_mode='categorical', subset='training', shuffle=True)
    val_gen = datagen.flow_from_directory(input_path, target_size=img_size, batch_size=batch_size,
                                          class_mode='categorical', subset='validation', shuffle=False)
    return train_gen, val_gen

# === Model Definition with Fine-Tuning ===
def create_densenet169_finetuned(input_shape, num_classes, fine_tune_at):
    base_model = DenseNet169(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=output)

# === Load Data ===
model_name = 'DenseNet169_FineTuned'
train_gen, val_gen = process_data(input_path, IMG_SIZE, BATCH_SIZE)
num_classes = train_gen.num_classes
class_names = list(train_gen.class_indices.keys())

# === Compile Model ===
model = create_densenet169_finetuned((*IMG_SIZE, 3), num_classes, FINE_TUNE_AT)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

# === Model Checkpoint ===
checkpoint_path = os.path.join(output_path, f'{model_name}_best_model.keras')
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# === Train Model ===
history = model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen, callbacks=[checkpoint])

# === Accuracy & Loss Plots ===
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title("Accuracy")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss")
plt.legend()
acc_loss_path = os.path.join(output_path, f'{model_name}_accuracy_loss.png')
plt.savefig(acc_loss_path)
plt.close()

# === Evaluation ===
model.load_weights(checkpoint_path)
eval_result = model.evaluate(val_gen)
Y_pred = model.predict(val_gen)
y_pred = np.argmax(Y_pred, axis=1)
y_true = val_gen.classes
report = classification_report(y_true, y_pred, target_names=class_names)

# === Confusion Matrix ===
cm = confusion_matrix(y_true, y_pred)
cm_path = os.path.join(output_path, f'{model_name}_confusion_matrix.png')
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xticks(rotation=45)
plt.title("Confusion Matrix")
plt.savefig(cm_path)
plt.close()

# === ROC Curve ===
y_true_bin = label_binarize(y_true, classes=range(num_classes))
fpr, tpr, roc_auc = {}, {}, {}
plt.figure(figsize=(10, 8))
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], Y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], label=f"{class_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Multi-Class")
plt.legend()
roc_path = os.path.join(output_path, f'{model_name}_roc_curve.png')
plt.savefig(roc_path)
plt.close()

# === Grad-CAM ===
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_channel = predictions[:, tf.argmax(predictions[0])]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# === Prepare Grad-CAM Image ===
img_path = val_gen.filepaths[0]
img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = tf.keras.applications.densenet.preprocess_input(img_array)

# Correct last conv layer for DenseNet169
last_conv_layer_name = "conv5_block16_concat"
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# === Overlay Grad-CAM ===
img_cv = cv2.imread(img_path)
heatmap_resized = cv2.resize(heatmap, (img_cv.shape[1], img_cv.shape[0]))
heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
superimposed = cv2.addWeighted(img_cv, 0.6, heatmap_color, 0.4, 0)

gradcam_path = os.path.join(output_path, f"{model_name}_gradcam.jpg")
cv2.imwrite(gradcam_path, superimposed)

# === Generate Word Report ===
doc.add_heading("Model Performance", level=1)
doc.add_paragraph(f"Final Validation Accuracy: {eval_result[1]:.4f}")
doc.add_paragraph(f"Final Validation Loss: {eval_result[0]:.4f}")
doc.add_heading("Classification Report", level=2)
doc.add_paragraph(report)
doc.add_picture(acc_loss_path, width=Inches(6))
doc.add_picture(cm_path, width=Inches(6))
doc.add_heading("ROC Curve", level=2)
doc.add_picture(roc_path, width=Inches(6))
doc.add_heading("Grad-CAM Visualization", level=2)
doc.add_picture(gradcam_path, width=Inches(6))
doc.save(os.path.join(output_path, f'{model_name}_report.docx'))

print("✅ DenseNet169 Fine-Tuned: Training, ROC, Grad-CAM, and report saved.")
